In [1]:
# Imports

import os

import csv

from selenium import webdriver
from requests import get
from bs4 import BeautifulSoup

In [2]:
# Statics
# Note: this URL works, but could probably be cleaned up

base_url = "http://bobcat.library.nyu.edu/primo_library/libweb/action/"
base_url += "search.do?fn=search&ct=search&initialSearch=true&mode=Basic&tab=all&indx=1&dum=true&srt=rank&vid=NYU&frbg=&vl%28freeText0%29="

infile = "data/isbns.txt"

In [3]:
# Read a txt file of isbns

with open(infile, "r") as f:
    isbns = f.read().splitlines()

In [4]:
# Function to validate isbns
# Should be other validation?

def validate_isbn(isbn):
    if len(isbn) == 13 or len(isbn) == 10:
        pass
    else:
        isbn = '0' * (10-len(isbn)) + isbn
    return isbn

In [5]:
# Fix leading zeroes

isbns = [validate_isbn(isbn) for isbn in isbns]

In [6]:
# Create browser instance
# Note: this requires Firefox to be installed

browser = webdriver.Firefox()

In [7]:
# Function for finding isbn matches in Bobcat (via Selenium)

def check_bobcat_isbn(isbn):
    check_bobcat = False # Set default return
    url = base_url + isbn # Build URL string
    browser.get(url) # Open url in browser instance; should trap response errors
    alert = browser.find_elements_by_class_name('alert') # Only missing ISBNs have an class called alert
    if len(alert) == 0:
        check_bobcat = True
    return check_bobcat

In [8]:
# Iterate over isbns and find matches

matches = []

for isbn in isbns:
    if check_bobcat_isbn(isbn):
        #print("Yes! %s is in the NYU catalog" % isbn)
        matches.append((isbn, True))
    else:
        #print("No! %s is not in the NYU catalog" % isbn)
        matches.append((isbn, False))

In [9]:
# Export to csv

with open('matches.csv','w') as out:
    csv_out=csv.writer(out, quotechar = "'")
    csv_out.writerow(['isbn','match'])
    for row in matches:
        csv_out.writerow(row)

In [10]:
# Close browser instance

browser.quit()